# Cross Sectional Mean Reversion Strategy

In [23]:
import yfinance as yf
import backtrader as bt
import numpy as np
import pandas as pd
import datetime

In [24]:
def download_stock(stock, start, end):    
    data = yf.download(stock, start, end)        
    return data 

In [25]:
class CrossSectionalMeanReversion(bt.Strategy):
    
    def __init__(self):
        self.stock_data =  self.datas
        
    def prenext(self):
        self.next()
        
    def next(self):
        
        stock_returns = np.zeros(len(stock_data))
        
        # calculate the last daily returns
        for index, stock in enumerate(self.stock_data):
            stock_returns[index] = (stock.close[0] -  stock.close[-1]) / stock.close[-1]
            
        # average return of the market (SP500)
        market_return = np.mean(stock_returns)
        # weights
        weights = -(stock_returns - market_return)
        weights = weights / np.sum(np.abs(weights))
        
        # we can update pur positions based on w weights
        for index, stock in enumerate(self.stock_data):
            self.order_target_percent(stock, target=weights[index])

In [26]:
if __name__ == "__main__":
    cerebro = bt.Cerebro()
    
    table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    df = table[0]
    df.to_csv('S&P500-Info.csv')
    df.to_csv("S&P500-Symbols.csv", columns=['Symbol'])
#    df = df[:10]   # First 10 stock to accelerate the code

    for line in df['Symbol']:
        start_date =  datetime.datetime(2010, 1, 1)
        end_date = datetime.datetime(2020, 1, 1)        
        stock_data = download_stock(line.strip("\n"), start_date, end_date)
#        stock_data.to_csv("data/"+str(line.strip("\n"))+'.txt', index=True)
                      
        try:
#            df = pd.read_csv("data/"+str(line.strip("\n"))+'.txt', parse_dates=True, index_col=0)
            df = pd.DataFrame(stock_data)
            if len(df) > 100:
                cerebro.adddata(bt.feeds.PandasData(dataname=df, plot=False))
        except FileNotFoundError:
            pass    
            
    cerebro.addobserver(bt.observers.Value)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.0)
    cerebro.addanalyzer(bt.analyzers.Returns)
    cerebro.addanalyzer(bt.analyzers.DrawDown)
    
    cerebro.addstrategy(CrossSectionalMeanReversion)
    
    cerebro.broker.set_cash(100000)
    print('Initial capital: $%.2f' % cerebro.broker.getvalue())
    
    results = cerebro.run()
    
    print(f"Sharpe: {results[0].analyzers.sharperatio.get_analysis()['sharperatio']:.3f}")
    print(f"Annual Return: {results[0].analyzers.returns.get_analysis()['rnorm100']:.2f}%")
    print(f"Max Drawdown: {results[0].analyzers.drawdown.get_analysis()['max']['drawdown']:.2f}%")
    print('Capital: $%.2f' % cerebro.broker.getvalue())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********